 # Mass and energy balances

Mass and energy balances are fundamental priciples for a chemical process. In this problem set, we will employ Stream objects to perform mass and energy balances for corn ethanol production at a conventional dry-grind plant.

![corn_dry_grind.png](../images/corn_dry_grind.png)

We will only model 4 steps:
    
    1. Mixing milled corn with water (32 % dry solids).
       - Find the amount of dilution water required.
       
    2. Cooking (mix with superheated steam to 210 degC).
       - Find the amount of steam required.
       
    3. Liquefaction (Starch + H2O -> Glucose at 87 degC).
       - Find the duty from step 2 to 3
       
    4. Fermentation (Glucose -> 2Ethanol + 2CO2 at 32 degC; 90 % theoretical yield).
       - Find the duty from step 3 to 4

**Instructions:** Replace all `None`s with the correct values.

Here are some assumptions you will need to use later:

In [1]:
solids_content = 0.32
T_cooking = 210 + 273.15 # K
T_steam = T_cooking + 10 # K
T_liquefaction = 87 + 273.15 # K
T_fermentation = 32 + 273.15 # K
liquefaction_yield = 1.0 # 100 %
fermentation_yield = 0.9 # 90 %

First, we need to define the chemicals. Let's assume the corn kernel is composed of starch (62%), fiber (19%), water (15%), and oil (4%). Use a basis of 40 MT/hr. Model the missing components, streams, and reactions in the code:

In [ ]:
import biosteam as bst

## Define chemicals ##

Water = None # Hint: bst.Chemical will load any chemical by name
Oil = bst.Chemical('Oil', search_ID='Tripalmitin', phase='l')
Starch = bst.Chemical(
    'Starch',
    search_db=False, # Do not search database
    phase='s', # Phase will not change even if stream is liquid
    formula="C6H10O5", # Glucose monomer minus water
    Cp=1.364, # Heat capacity [J / g / K]
    Hf=-975709, # Heat of formation [J / mol]
    default=True, # Default the rest
)
Fiber = None # Hint: Define fiber with the same asssumptions as starch
Glucose = bst.Chemical('Glucose', phase='s')
bst.settings.set_thermo([Water, Starch, Fiber, Oil, Glucose, 'Ethanol', 'CO2'])

## Create feed streams ##

steam = bst.Stream(
    phase='g', T=508.99, P=3.11e+06,
    Water=1, units='kmol/hr'
)
dilution_water = None # Hint: Create an empty stream, the flow rate will be adjusted later.
corn = None # Hint: Use units='MT/hr'

## Create reactions ##

liquefaction = bst.Reaction(
    'Starch + Water -> Glucose', reactant='Starch', X=liquefaction_yield
)
fermentation = None # Hint: Follow example above

Finally we can perform the mass and energy balances for this problem:

**Step 1: Mixing mass balance**

$$ F_{water} = \frac{1 - x_{solids}}{x_{solids}} F_{solids} $$
$$ F_{water} = F_{moisture} + F_{dilution\ water} $$
$$ F_{dilution\ water} = \frac{1 - x_{solids}}{x_{solids}} F_{solids} -  F_{moisture}$$

In [ ]:
import numpy as np
from numpy.testing import assert_allclose

F_moisture = corn.imass['Water']
F_solids = (corn.F_mass - F_moisture) 
dilution_water.imass['Water'] = None # Hint: F_dilution_water is given by the formula
slurry = bst.Stream.sum([corn, dilution_water], ID='slurry')
assert_allclose(slurry.get_mass_fraction('Water'), 1 - solids_content)

**Step 2: Cooking energy balance**

$$ H - \text{Enthalpy flow}\ [kJ \cdot hr^{-1}] $$
$$ h - \text{Specific enthalpy}\ [kJ \cdot kmol^{-1}] $$

$$ H_{slurry}(T_{slurry}) + h_{steam}(T_{steam}) F_{steam} = H_{slurry}(T_{cooking}) + h_{water}(T_{cooking}) F_{steam} $$

$$ F_{steam} = \frac{H_{slurry}(T_{slurry}) - H_{slurry}(T_{cooking})}{h_{water}(T_{cooking}) - h_{steam}(T_{steam})} $$

In [ ]:
water = steam.copy('water')
water.phase = 'l'
hot_slurry = slurry.copy('hot_slurry')
hot_slurry.T = water.T = T_cooking
hot_slurry.P = water.P # No back flow
H_cooking = hot_slurry.H
H_slurry = None
h_cooking = water.h
h_steam = None # Hint: Use the steam stream
steam.imol['Water'] = None # Hint: Use energy balance above
cooked_slurry = slurry + steam
assert_allclose(cooked_slurry.T, T_cooking)

**Step 3: Liquefaction reaction**

$$ Starch + Water \rightarrow Glucose $$

In [ ]:
liquified_slurry = cooked_slurry.copy()
liquified_slurry.T = T_liquefaction
liquefaction(None) # Reaction alters stream flow rates isothermally
assert_allclose(liquified_slurry.imol['Glucose'], cooked_slurry.imol['Starch'])

**Step 4: Fermentation reaction**

$$ Glucose \rightarrow 2Ethanol + 2CO_2 $$

In [ ]:
fermentation_broth = None # Hint: follow steps in liquifaction
fermentation(None)
assert_allclose(fermentation_broth.imass['Ethanol', 'CO2'].sum(), liquified_slurry.imass['Glucose'])
fermentation_broth.vle(T=T_fermentation, P=101325)
fermentation_broth.show('cwt')

Let's compute our answers:

In [ ]:
print(f'Amount of dilution water required: {dilution_water.F_mass / corn.F_mass:.3g} kg')
print(f'Amount of steam required: {steam.F_mass / corn.F_mass:.3g} kg')
print(f'Duty from step 2 to 3: {(liquified_slurry.H - cooked_slurry.H) / corn.F_mass:.3g} kJ')
print(f'Duty from step 3 to 4: {(fermentation_broth.H - liquified_slurry.H) / corn.F_mass: .3g} kJ')